In [ ]:
# Notebook to create subsampled (e.g. 75%, 50%, 25%, 10%, 5%, 1%, 0.1%) of training data to evaluate
# training in the low-data setting

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
join = os.path.join
from tqdm import tqdm
from skimage import transform 
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
import sys
sys.path.append('./modified_medsam_repo')
import random
from datetime import datetime
import pandas as pd
import nibabel as nib
import pickle

In [2]:
full_df = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_constant_bbox.csv')
splits = pd.read_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/train_val_test_split.pickle')

In [17]:
# first pass
percents_interested = [100, 75, 50, 25, 10, 5, 2.5, 1, 0.5, 0.1]

src_path = '/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled'
np.random.seed(2024)

last_train_list = splits['train']
for xo in percents_interested:
    mult = xo / 100.0
    num_train_to_sample = round(len(splits['train']) * mult)
    
    train_idxs = np.random.choice(last_train_list, size = num_train_to_sample, replace=False)
    new_dict = {'train': train_idxs, 'val': splits['val'], 'test':splits['test']}
    

    pd.to_pickle(new_dict, f'{src_path}/{xo}.pkl')
    #print(train_idxs)
    last_train_list = train_idxs

In [4]:
len(splits['train'])

891

In [14]:
0.75 * 891

668.25

In [15]:
np.random.seed(2024)
num_mri_interested = [1, 2, 3, 4, 5, 7, 10, 15, 20, 50, 100, 250, 500, 891]
src_path = '/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled/repeat_experiments'
repeats = 10
for num in num_mri_interested:
    targ_path = os.path.join(src_path, str(num))
    if not os.path.exists(targ_path):
        os.makedirs(targ_path)
    
    for i in range(repeats):
        train_idxs = np.random.choice(splits['train'], size = num, replace=False)
        new_dict = {'train': train_idxs, 'val': splits['val'], 'test': splits['test']}
        final_path = os.path.join(targ_path, f'repeat{i}.pkl')
        pd.to_pickle(new_dict, final_path)



In [31]:
pd.read_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/medsam_retrain_dice_batchsz_expt_11-8-24/val/1/0.0005/0.1/8/eval_singletask_unprompted_val_label1.pkl')

{'dice_sensitivity': tensor([0.7936]),
 'dice_specificity': tensor([0.6686]),
 'overall_dice': tensor([0.7211]),
 'label_numbers': []}

In [33]:
pd.read_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/results_copied_from_kn2347/unet_retrain_dice_batchsz_expt_11-11-24/val/0.0005/1/0.1/256/eval_singletask_unet_val_label1.pkl')

{'dice_sensitivity': tensor([0.7839]),
 'dice_specificity': tensor([0.8372]),
 'overall_dice': tensor([0.8148]),
 'label_numbers': []}